In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import numpy as np

/home/jupyter/MedClip/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pipelines import build_pipeline
pipeline=build_pipeline()
pipeline.keys()

2023-03-01 05:19:55.197779: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 05:19:55.284103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 05:19:55.285933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 05:19:55.319541: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

dict_keys(['captioning'])

In [3]:
pipeline['captioning'].keys()
dict_keys(['train', 'val', 'test', 'train_captions'])

NameError: name 'dict_keys' is not defined

In [ ]:
import download as dl

In [ ]:
dl.medpix()

In [ ]:
df_subset=pd.read_csv('../data/intermediate/inter_medpix.csv')

In [ ]:
def decode_and_resize(img_path,image_size):
    """Recieves an image path, decodes and rescales it. """
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, image_size)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def to_dict(image, text):
    """ To be called on dataset.map to get a dictionary"""
    return {'image': image, 'text': text}

def build_tf_dataset(df_subset,image_size):
    """Build captioning tensorflow dataset from dataframe
    
    The captioning tensorflow dataset returns dictionaries
    containing "images" and "text" keys.
    
    Args: 
        df_subset(pandas df): The dataframe from which to extract
            imagepaths and captions. It must contain the columns
            "Full_Caption" and "Path"
    Returns
        captioning_tf_dataset(tf dataset): Captioning tensorflow dataset. 
    """
    # Build images tf dataset
    images_tf_dataset = (tf.data.Dataset.from_tensor_slices(df_subset['Path'])
                            .map(lambda x: decode_and_resize(x,image_size),num_parallel_calls=tf.data.AUTOTUNE)
                        )

    # Build captions tf_dataset
    captions_tf_dataset = (tf.data.Dataset
                              .from_tensor_slices(df_subset['Full_Caption'].astype(str))
                              .map(lambda x:tf.expand_dims(x,axis=0))
                          )

    # Zip df_subset images and captions datasets into a captioning dataset. 
    captioning_tf_dataset = tf.data.Dataset.zip(
                                                    (images_tf_dataset, 
                                                     captions_tf_dataset)
                                                  ).map(to_dict)
    return captioning_tf_dataset


In [ ]:
tf.data.Dataset.from_tensor_slices(df_subset['Path'])

In [ ]:
image_size=(299,299)
images_tf_dataset = (tf.data.Dataset.from_tensor_slices(df_subset['Path'])
                            .map(lambda x: decode_and_resize(x,image_size),num_parallel_calls=tf.data.AUTOTUNE)
                        )
captions_tf_dataset = (tf.data.Dataset
                              .from_tensor_slices(df_subset['Full_Caption'].astype(str))
                              .map(lambda x:tf.expand_dims(x,axis=0))
                          )
paths_tf_dataset = (tf.data.Dataset.from_tensor_slices(df_subset['Path']))
                    
captioning_tf_dataset = tf.data.Dataset.zip({'images':images_tf_dataset, 
                                             'text':captions_tf_dataset,
                                             'paths':paths_tf_dataset})

In [ ]:
for i in captioning_tf_dataset.take(1):
    print(i)

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices({'images':images_tf_dataset,
                                    'captions':captions_tf_dataset})

In [ ]:


wb = openpyxl.load_workbook('images.xlsx')
ws = wb.get_sheet_by_name('Daniel1')
ws.column_dimensions['H'].width = 50
ws.row_dimensions['5'].height = 50
img = openpyxl.drawing.image.Image('test-fig.png')
img.anchor = 'H5'
ws.add_image(img)
wb.save('images.xlsx')


In [ ]:
build_pipeline()

In [ ]:
# %load_ext autoreload
# %autoreload 2
from pipelines import build_pipeline, build_coco_pipeline
# import tensorflow as tf
# import re
# import numpy as np
# import matplotlib.pyplot as plt
# import time
# import datetime
# import textwrap
# import json
# import pandas as pd
# import os
# import pickle
# # Tokenizers
# from tokenizers import KerasTokenizer
# # Model
# from models.CaptioningTransformers import CaptioningTransformer
# from models.Callbacks import CaptioningCallback
# from models.model_utils import load_captioning_model
# # Results
# from prediction import Predict
# from tqdm import tqdm
# from evaluation import Evaluate
# from models.OldCaptioningTransformer import instantiate_captioner
# import seaborn as sns

In [ ]:
# from experiment import Experiment, AblationStudy, ExperimentCluster
# config_1={
#     'name':'model_big',
#     'dataset':'medpix',
#     'model_name':'exp_3',
#     'seq_len':178,
#     'num_heads':8,
#     'model_dim':512,
#     'dropout':0.2,
#     'img_backbone_trainable':False,
#     'img_size':(299,299,3),
#     'img_backbone':'efficientnetb0',
#     'intermediate_size':512*4,
#     'vocab_size':None,
#     'capts_per_img':1,
#     'num_encoder_blocks':6,
#     'num_decoder_blocks':6,
#     'max_epochs':2,
#     'learning_rate':0.001,
#     'normalization':'pre',
#     'metric':'val_acc'
# }

# config_2={
#     'name':'model_mid',
#     'dataset':'medpix',
#     'model_name':'exp_3',
#     'seq_len':178,
#     'num_heads':4,
#     'model_dim':256,
#     'dropout':0.2,
#     'img_backbone_trainable':False,
#     'img_size':(299,299,3),
#     'img_backbone':'efficientnetb0',
#     'intermediate_size':256*4,
#     'vocab_size':None,
#     'capts_per_img':1,
#     'num_encoder_blocks':3,
#     'num_decoder_blocks':3,
#     'max_epochs':2,
#     'learning_rate':0.001,
#     'normalization':'pre',
#     'metric':'val_acc'
# }


# config_3={
#     'name':'model_small',
#     'dataset':'medpix',
#     'model_name':'exp_3',
#     'seq_len':178,
#     'num_heads':1,
#     'model_dim':128,
#     'dropout':0.2,
#     'img_backbone_trainable':False,
#     'img_size':(299,299,3),
#     'img_backbone':'efficientnetb0',
#     'intermediate_size':128*4,
#     'vocab_size':None,
#     'capts_per_img':1,
#     'num_encoder_blocks':1,
#     'num_decoder_blocks':1,
#     'max_epochs':2,
#     'learning_rate':0.001,
#     'normalization':'pre',
#     'metric':'val_acc'
# }

# configs=[config_1,config_2,config_3]
# experiment_cluster=ExperimentCluster(configs,'../models/cluster_1')
# experiment_cluster.run()


In [ ]:
# experiment_cluster.results

In [4]:
from evaluation import Evaluate

In [5]:
ev = Evaluate('../models/study_3_medpix_dividedcases/0')

In [6]:
ev.confusion_matrixes()